**Import Pyspark**

In [122]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StringType,IntegerType,StructType
from pyspark.sql import functions as f

**Start Spark**

In [123]:
spark = SparkSession.builder.appName("SemantixChallenge").getOrCreate()

**Create Schema**

In [124]:
data_schema = [StructField("host", StringType(), True),StructField("timestamp", StringType(), True),
              StructField("timezone", StringType(), True),StructField("request", StringType(), True),
              StructField("type_request", StringType(), True),StructField("response", StringType(), True),
              StructField("bytes", StringType(), True)]

In [125]:
type (data_schema)

list

In [126]:
final_struc = StructType(fields=data_schema)

**Load Data**

In [127]:
df = spark.read.csv('NASA_access_log_merged_Jul95_Aug95.csv', schema=final_struc)

**Show Schema**

In [128]:
df.printSchema()

root
 |-- host: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- request: string (nullable = true)
 |-- type_request: string (nullable = true)
 |-- response: string (nullable = true)
 |-- bytes: string (nullable = true)



**Show Data**

In [129]:
df.show(10)

+--------------------+--------------------+--------+--------------------+------------+--------+-----+
|                host|           timestamp|timezone|             request|type_request|response|bytes|
+--------------------+--------------------+--------+--------------------+------------+--------+-----+
|        199.72.81.55|01/Jul/1995:00:00:01|   -0400| GET/history/apollo/|    HTTP/1.0|     200| 6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00:06|   -0400|GET/shuttle/count...|    HTTP/1.0|     200| 3985|
|      199.120.110.21|01/Jul/1995:00:00:09|   -0400|GET/shuttle/missi...|    HTTP/1.0|     200| 4085|
|  burger.letters.com|01/Jul/1995:00:00:11|   -0400|GET/shuttle/count...|    HTTP/1.0|     304|    0|
|      199.120.110.21|01/Jul/1995:00:00:11|   -0400|GET/shuttle/missi...|    HTTP/1.0|     200| 4179|
|  burger.letters.com|01/Jul/1995:00:00:12|   -0400|GET/images/NASA-l...|    HTTP/1.0|     304|    0|
|  burger.letters.com|01/Jul/1995:00:00:12|   -0400|GET/shuttle/count...|    HTTP/

**1) Number of unique hosts.**

In [130]:
df.select("host").distinct().count()

137979

**R**:The number of unique hosts is 137979

**2) Total 404 errors.**

In [131]:
df.filter('response=404').count()

20634

**R:** The total number of 404 errors is 20634

**3) Os 5 URLs que mais causaram erro 404.**

In [132]:
x = df.filter('response=404')


In [133]:
hosts = x.groupBy('host').count()

In [134]:
hosts.orderBy(hosts['count'].desc()).show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc.edu|  251|
|piweba3y.prodigy.com|  157|
|jbiagioni.npt.nuw...|  132|
|piweba1y.prodigy.com|  114|
|www-d4.proxy.aol.com|   91|
|piweba4y.prodigy.com|   86|
|scooter.pa-x.dec.com|   69|
|phaelon.ksc.nasa.gov|   64|
|www-d1.proxy.aol.com|   64|
|dialip-217.den.mm...|   62|
|www-b4.proxy.aol.com|   62|
|www-b3.proxy.aol.com|   61|
|www-a2.proxy.aol.com|   60|
|www-d2.proxy.aol.com|   59|
|piweba2y.prodigy.com|   59|
|            titan02f|   59|
|monarch.eng.buffa...|   56|
|  alyssa.prodigy.com|   56|
|www-c4.proxy.aol.com|   53|
|www-b2.proxy.aol.com|   53|
+--------------------+-----+
only showing top 20 rows



**R**:The five hosts that had the most 404 errors were hoohoo.ncsa.uiuc.edu, piweba3y.prodigy.com, jbiagioni.npt.nuw, piweba1y.prodigy.com and www-d4.proxy.aol.com

**4) Number of 404 errors per day.**

In [108]:
x = df.filter('response=404')

In [112]:
day = x.groupBy('timestamp').count()

In [116]:
day.orderBy(day['count'].desc()).show(30)
#hosts.orderBy(hosts['count'].desc()).show()


+--------------------+-----+
|           timestamp|count|
+--------------------+-----+
|28/Aug/1995:11:56:35|    7|
|11/Aug/1995:12:05:59|    7|
|12/Jul/1995:10:21:30|    5|
|12/Jul/1995:10:20:43|    5|
|28/Aug/1995:17:14:32|    5|
|28/Aug/1995:17:14:42|    5|
|12/Jul/1995:10:35:11|    5|
|12/Jul/1995:10:24:50|    5|
|11/Aug/1995:12:05:58|    5|
|12/Jul/1995:10:35:09|    5|
|11/Jul/1995:14:08:06|    5|
|12/Jul/1995:10:35:12|    5|
|17/Aug/1995:16:55:00|    5|
|06/Jul/1995:10:31:11|    4|
|04/Aug/1995:18:45:58|    4|
|28/Aug/1995:17:14:15|    4|
|28/Aug/1995:01:05:47|    4|
|28/Aug/1995:17:14:28|    4|
|13/Aug/1995:14:28:59|    4|
|12/Jul/1995:10:35:01|    4|
|20/Jul/1995:07:21:17|    4|
|12/Jul/1995:10:35:03|    4|
|12/Jul/1995:10:21:32|    4|
|11/Jul/1995:14:08:20|    3|
|20/Jul/1995:03:29:20|    3|
|12/Jul/1995:10:23:35|    3|
|12/Jul/1995:10:21:28|    3|
|12/Jul/1995:10:20:44|    3|
|06/Jul/1995:10:31:12|    3|
|14/Aug/1995:15:06:49|    3|
+--------------------+-----+
only showing t

In [117]:
avg = (7+5+4+3) / 3
print (avg)

6.333333333333333


**R:** the average of 404 errors per day, in the period of July and August is 6 per day

**5) The total number of bytes returned.**

In [120]:
df.groupBy().agg(f.sum('bytes')).show()

+---------------+
|     sum(bytes)|
+---------------+
|6.5155182365E10|
+---------------+



**R:** The sum of bytes returned is 6,5155182365 bytes, which is equivalent to 65.1 Gigabytes.